## Character-level text generation using LSTMs

Before we have seen how to generate text using Markov models. In this tutorial, we will see how can we do it using RNNs. 

LSTMs are expensive to train and, I recommend that you run this on the cloud using [Google colab](https://colab.research.google.com/notebooks/welcome.ipynb). This will allow you to train on a GPU and assess the benefits of training neural networks on GPUs.

You will follow the steps below.

- Go to [Google colab](https://colab.research.google.com/). 
- Make an account if you don't have one.
- Select "UPLOAD" and upload this notebook. 
- Runtime --> change runtime type --> Select GPU.
- Run the notebook. 


## Setup 

In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os, gc

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM

## Read the data

I have created a toy corpus in the students directory and we'll be working with this corpus in this demonstration. 

In [0]:
from urllib.request import urlopen
data_url = 'https://raw.github.ubc.ca/MDS-2019-20/DSCI_575_adv-mach-learn_students/master/lectures/data/toy_cohen_poetry.txt?token=AAAANPYW3MZ66UPOTCMGER26UZKPM'
text = urlopen(data_url).read().decode("utf-8")

In [4]:
# Take a look at the first 250 characters in text
text = text.lower()
print(text[:250])

the birds they sang
at the break of day
start again
i heard them say
don't dwell on what
has passed away
or what is yet to be
yeah the wars they will
be fought again
the holy dove
she will be caught again
bought and sold
and bought again
the dove is 


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

34 unique characters


## Preprocessing

Let's prepare our text so that we can feed it into an RNN.

### Vectorize the text

- Map unique characters to indices. 
- Represent a sequence of characters as a sequence of indices.

In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [7]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '"' :   2,
  "'" :   3,
  '(' :   4,
  ')' :   5,
  ',' :   6,
  '.' :   7,
  '?' :   8,
  'a' :   9,
  'b' :  10,
  'c' :  11,
  'd' :  12,
  'e' :  13,
  'f' :  14,
  'g' :  15,
  'h' :  16,
  'i' :  17,
  'j' :  18,
  'k' :  19,
  ...
}


In [0]:
n_vocab = len(vocab)

### Create `X` and `y`

Our prediction task is: Given or a sequence of characters, what is the most likely character? Remember that in RNNs, we maintain the hidden state that maintains information about all the input we have seen so far. 

- So first design decision is how long we want to maintain this information. We define a `seq_length` and remember the information for that much time. You can roughly think of this as `n` in your n-gram model.

- So `X` is going to be sequence of characters and `y` is going to be the next character. 



With the LSTM, the data matrix `X` is now 3-dimensional instead of 2-dimensional. The dimensions are

- number training examples
- sequence length (this is the `n`  of the  `n-grams`)
- number of features, which in this case is a 1-hot encoding of the  𝑑  characters in our vocabulary

In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 25
n_examples = len(text) - seq_length

# Let's create X and y
data_X = np.zeros((n_examples, seq_length, n_vocab),dtype=bool)
data_y = np.zeros((n_examples, n_vocab))

count = 10    
for i in range(n_examples):
    seq_in = text[i:i + seq_length]
    char_out = text[i + seq_length]
    for j, char in enumerate(seq_in):
        data_X[i, j, char2idx[char]] = 1.0
    data_y[i, char2idx[char_out]] = 1.0


In [10]:
# Let's look at X
data_X

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False,  True, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False,  True, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False,  True, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, Fal

In [11]:
# Let's look at y
data_y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [12]:
# How many examples do we have? 
print('Total examples: %d'%(len(data_X)))
print('Total examples: %d'%(len(data_y)))

Total examples: 4419
Total examples: 4419


## Build the model

Let's build an LSTM model with just one layer. 

In [0]:
def create_model(seq_length, 
                 n_vocab, 
                 layer_size=256, 
                 dropout_amount=0.5):
    model = Sequential()
    model.add(LSTM(layer_size, input_shape=(seq_length, n_vocab), 
                   return_sequences=True))
    model.add(Dropout(dropout_amount))
    model.add(LSTM(layer_size, return_sequences=True))
    model.add(Dropout(dropout_amount))
    model.add(LSTM(layer_size))
    model.add(Dropout(dropout_amount))
    model.add(Dense(n_vocab, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model


In [14]:
n_examples, seq_length, n_vocab = data_X.shape

model = create_model(seq_length, n_vocab)

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 25, 256)           297984    
_________________________________________________________________
dropout (Dropout)            (None, 25, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 25, 256)           525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 25, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 34)                8

In [15]:
# fit the model

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

model.fit(data_X, data_y, 
         epochs=100, 
         batch_size=128, 
         callbacks=[checkpoint_callback], 
         validation_split=0.20)

Epoch 1/100
28/28 [==============================] - 2s 58ms/step - loss: 3.1666 - accuracy: 0.1330 - val_loss: 2.9809 - val_accuracy: 0.1618
Epoch 2/100
28/28 [==============================] - 0s 18ms/step - loss: 3.0233 - accuracy: 0.1562 - val_loss: 2.9578 - val_accuracy: 0.1618
Epoch 3/100
28/28 [==============================] - 0s 17ms/step - loss: 3.0136 - accuracy: 0.1610 - val_loss: 2.9875 - val_accuracy: 0.1618
Epoch 4/100
28/28 [==============================] - 0s 17ms/step - loss: 2.9990 - accuracy: 0.1595 - val_loss: 2.9627 - val_accuracy: 0.1618
Epoch 5/100
28/28 [==============================] - 0s 18ms/step - loss: 2.9836 - accuracy: 0.1593 - val_loss: 2.9510 - val_accuracy: 0.1618
Epoch 6/100
28/28 [==============================] - 0s 18ms/step - loss: 2.9730 - accuracy: 0.1567 - val_loss: 2.9486 - val_accuracy: 0.1618
Epoch 7/100
28/28 [==============================] - 0s 17ms/step - loss: 2.9496 - accuracy: 0.1610 - val_loss: 2.9675 - val_accuracy: 0.1527
Epoch 

## Generate text

In [0]:
import sys
def generate(model, seed, n_vocab, int_to_char, temperature=1, num_gen=1000):
    print ("Seed:")
    for p in seed:
        sys.stdout.write(int_to_char[np.argmax(p)])
    print("\n--------------------------------------------\n")
    pattern = seed

    # generate characters
    for i in range(num_gen):
        prediction = model.predict(pattern[None], verbose=0)

        probabilities = prediction.flatten()
        if temperature != 1:
            # maybe not exactly right but close enough, and probably exactly right
            probabilities = probabilities**(1.0/temperature) 
            probabilities /= np.sum(probabilities)

        index = np.random.choice(n_vocab, p=probabilities)
            
        result = int_to_char[index]
        sys.stdout.write(result)
        sys.stdout.flush()

        new_char_one_hot = np.zeros(n_vocab)
        new_char_one_hot[index] = 1.0
        pattern = np.append(pattern[1:], new_char_one_hot[None], axis=0)
        
    print("\n")
    gc.collect() # http://stackoverflow.com/questions/40560795/tensorflow-attributeerror-nonetype-object-has-no-attribute-tf-deletestatus

In [18]:
# pick random seed (one of two ways depending on primetext)
import gc
start = np.random.randint(0, n_examples)
seed = data_X[start]
generate(model, seed, n_vocab, idx2char, temperature=0.5, num_gen=500)

Seed:
id played and it pleased 
--------------------------------------------

the lord
but you don't really care for music, do you?
well it goes like this
the fourth, the fifth, the minor fall and the major lift
the baffled king composing hallelujah
hallelujah
helllujah
hsssluaah wyas bow tillhw the bboolrc hin
thd bofodl  ang termswiaig the lollls
the shil hhee tas iing
orgt hell bfrnn
th be fon lon the sror win in toe belll) that stil clae ring
forget your prffee toffering
there is  acrccc, a akac ii eveeyhhin tter iir mrf
fienht teeelisw wo si, ioney louctedd ago ai
ia

